### Notice:
In the function page_control(), the method.getProperty('textContent') is not well understood of its usage;

In this module, this method returned plain text result rather than html formatted text;

So maybe next time you use this module, you should pay attention to this line of code.

### propertyName includes at least:
- 'href'
- 'textContent'
- 'title'

---
### method.jsonValue():
coroutine jsonValue() → <u>***Dict***</u>[KT, VT]
    Get Jsonized value of this object.

In [ ]:
# Try to look up 'deposit values' when requesting HTML with pyppeteer
import nest_asyncio
nest_asyncio.apply()
import asyncio
from pyppeteer import launch
import pandas as pd
import time

def create_lst_task_url(dataframe, lst_tsk_len:int):
    '''
    : param lst_tsk_len: notify that it determines concurrently running numbers of tasks
    '''
    homepage = 'https://jpxkc.cbex.com'
    srs_url = dataframe['deep_url']
    srs_tit = dataframe['title']
    lst_task_url = []
    lst_title = []
    for i in range(len(srs_url)//lst_tsk_len):
        imlst_tar = []
        imlst_2 = []
        for k in range(lst_tsk_len):
            imlst_tar.append(homepage + srs_url[i * lst_tsk_len + k])
            imlst_2.append(srs_tit[i * lst_tsk_len + k])
        lst_task_url.append(imlst_tar)
        lst_title.append(imlst_2)
    imlst_tar = []
    imlst_2 = []
    for i in range(len(srs_url) - len(srs_url)%lst_tsk_len, len(srs_url)):
        imlst_tar.append(homepage + srs_url[i])
        imlst_2.append(srs_tit[i])
    lst_task_url.append(imlst_tar)
    lst_title.append(imlst_2)
    return lst_task_url, lst_title

async def page_control(url, browser):
    '''
    Notify: method.getProperty() should be paid attention to;
    '''
    page = await browser.newPage()
    await page.setJavaScriptEnabled(enabled=True)
    await page.setUserAgent(
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
        '(KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36 Edge/16.16299')
    xpathPromise = asyncio.ensure_future(page.waitForXPath('/html/body/div[6]/div[2]/div[1]/div[2]'))
    print(f'\r{time.perf_counter()-t0:.3f}s',end='')
    try: # Notify waitForXPath timeout default 30 sec => raise Exception
        await page.goto(url)  # indirectly cause a navigation(?)
        await xpathPromise  # Wait until element which matches xpath appears on page
    except TimeoutError:
        result = 'TimeoutError'
    else:
        result = await (await xpathPromise.result().getProperty('textContent')).jsonValue()
    await page.close()
    print(f'\r{time.perf_counter()-t0:.3f}s',end='')
    return result

async def browser_control(lst_task_url):
    '''
    lst_task_url should be a 2D url list; len of second dimens defines concurrently running no.

    ATTENTION: Please manually close the chromium.
    '''
    browser = await launch(
        {
            'headless': False, 
            'userDataDir': r'D:/haruk/Documents/PythonCodes/Temp', 
            'args': ['--no-sandbox'], 
            'dumpio': True
            }
            )
    # encapsule coroutine (lst_task)
    lst_task = []
    for item in lst_task_url:
        imlst = []
        for subitem in item:
            imlst.append(page_control(url=subitem, browser=browser))
        lst_task.append(imlst)
    # execute tasks
    lst_result = []
    for i in range(len(lst_task)):
        print(f'\r{time.perf_counter()-t0:.3f}s',end='')
        result = await asyncio.gather(*lst_task[i])
        lst_result.append(result)
        print(f'\r{time.perf_counter()-t0:.3f}s',end='')

    return lst_result

def main():
    dtfm_car = pd.read_pickle('')
    lst_task_url, lst_title = create_lst_task_url(dataframe=dtfm_car, lst_tsk_len=5)
    
    global t0
    t0 = time.perf_counter()
    print('Task Starts')

    loop = asyncio.get_event_loop()

    lst_res = loop.run_until_complete(browser_control(lst_task_url))

    print(f'\nDownload complete: {time.perf_counter()-t0:.4f}s')

    lst_sv = []
    for i in range(len(lst_res)):
        for k in range(len(lst_res[i])):
            lst_sv.append([lst_title[i][k], lst_res[i][k]])

    return lst_sv

'''
TEST
'''
if __name__=='__main__':
    lst_result = main()

In [ ]:
def search_tar(lst_result):
    import re
    lst_tar = []
    for i in range(len(lst_result)):
        text = lst_result[i][1]

        tara = re.search(r'最高限价报价人数：\d+人', text)
        if tara:
            tara = tara.group(0)[9:-1]
        tarb = re.search(r'摇号申请编码：\d{13}', text)
        if tarb:
            tarb = tarb.group(0)[7:]
        tarc = re.search(r'摇号次数：\d{2}', text)
        if tarc:
            tarc = tarc.group(0)[5:]
        tard = re.search(r'摇号注册时间：20\d\d-\d\d-\d\d', text)
        if tard:
            tard = tard.group(0)[7:]

        lst_tar.append([lst_result[i][0],tara,tarb,tarc,tard])
    return lst_tar

def result_check(lst_tar):
    chklst = []
    for i in range(len(lst_tar)):
        if any([
            lst_tar[i][1] == '',
            lst_tar[i][2] == None,
            lst_tar[i][3] == None,
            lst_tar[i][2] == '',
            lst_tar[i][3] == '',
        ]):
            chklst.append(i)
    print(len(chklst))
    return chklst

lst_tar = search_tar(lst_result)
chklst = result_check(lst_tar)